<a href="https://colab.research.google.com/github/PritamMoitra/Learning_Python_for_Finance/blob/main/Price_VolumeTradeSignals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import os

In [ ]:
symbols = ['SAIL.NS']
start_date = '2025-06-10'
end_date = '2025-06-11'
sma_period = 20
vwma_period = 20
initial_capital = 50000
inte='5m'

In [ ]:
# Output directory for CSVs (optional)
output_dir = "trading_signals"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def fetch_data(symbol, start_date, end_date,inte):
    return yf.download(symbol, start=start_date, end=end_date,interval=inte)

In [ ]:
def compute_indicators(df, sma_period, vwma_period):
    df['SMA'] = df['Close'].rolling(window=sma_period).mean()

    pv = df['Close'] * df['Volume']
    vwma = pv.rolling(window=vwma_period).sum() / df['Volume'].rolling(window=vwma_period).sum()
    df['VWMA'] = vwma

    return df

In [ ]:
# Signal generation function
def generate_signals(df, vwma_period=20):
    df = df.copy()  # Avoid modifying original
    df.dropna(subset=['Close', 'Volume'], inplace=True)  # Ensure both have valid data

    df['SMA'] = df['Close'].rolling(window=vwma_period).mean()

    price_volume = df['Close'] * df['Volume']
    vwma_numerator = price_volume.rolling(window=vwma_period).sum()
    vwma_denominator = df['Volume'].rolling(window=vwma_period).sum()

    df['VWMA'] = vwma_numerator / vwma_denominator

    df['Signal'] = 0
    df.loc[df['Close'] > df['VWMA'], 'Signal'] = 1
    df.loc[df['Close'] < df['VWMA'], 'Signal'] = -1
    df['Position'] = df['Signal'].diff()

    return df

In [ ]:
def plot_cumulative_returns(df, symbol):
    plt.figure(figsize=(14,6))
    plt.plot(df['Cumulative Market Return'], label='Market Return', linestyle='--')
    plt.plot(df['Cumulative Strategy Return'], label='Strategy Return')
    plt.title(f"{symbol} Strategy vs Market Returns")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{symbol}_returns.png"))
    plt.close()

In [ ]:
# Plotting functions
def plot_signals(df, symbol):
    plt.figure(figsize=(14,6))
    plt.plot(df['Close'], label='Close Price', alpha=0.5)
    plt.plot(df['SMA'], label='SMA', linestyle='--')
    plt.plot(df['VWMA'], label='VWMA', linestyle='--')
    buy_signals = df[df['Position'] == 2]
    sell_signals = df[df['Position'] == -2]
    plt.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy Signal', alpha=1)
    plt.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell Signal', alpha=1)
    plt.title(f"{symbol} Price with Buy/Sell Signals")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{symbol}_signals.png"))
    plt.close()

In [ ]:
# Return computation
def compute_simple_returns(df):
    df['Market Return'] = df['Close'].pct_change()
    df['Strategy Return'] = df['Market Return'] * df['Signal'].shift(1)
    df['Cumulative Market Return'] = (1 + df['Market Return']).cumprod()
    df['Cumulative Strategy Return'] = (1 + df['Strategy Return']).cumprod()
    return df

In [ ]:
# Trade log function
def generate_trading_log(df, symbol, starting_capital=50000):
    if 'Position' not in df.columns or df.empty:
        return pd.DataFrame(), starting_capital

    capital = starting_capital
    position = 0
    entry_price = 0
    log = []

    for date, row in df.iterrows():
        signal = row['Position']
        price = row['Close']

        if signal == 2 and position == 0:
            entry_price = price
            position = 1
            log.append({'Date': date, 'Symbol': symbol, 'Action': 'BUY',
                        'Price': round(price, 2), 'Capital Before': round(capital, 2),
                        'Capital After': round(capital, 2)})
        elif signal == -2 and position == 1:
            profit = price - entry_price
            capital += profit
            position = 0
            log.append({'Date': date, 'Symbol': symbol, 'Action': 'SELL',
                        'Price': round(price, 2), 'Capital Before': round(capital - profit, 2),
                        'Capital After': round(capital, 2)})

    return pd.DataFrame(log), capital

In [ ]:
# --------------------------------------------------------
# MAIN LOOP
# --------------------------------------------------------

final_capital = initial_capital
all_logs = []

required_columns = {'Close', 'Volume'}

for symbol in symbols:
    print(f"\nProcessing: {symbol}")
    df = fetch_data(symbol, start_date, end_date,inte)
    if isinstance(df.columns, pd.MultiIndex):
      try:
        df = df.xs(symbol, level=1, axis=1)
      except Exception as e:
        print(f"Failed to extract data for {symbol}: {e}")
        continue
    print(df.head(1))
    df = compute_indicators(df, sma_period, vwma_period)

    if df.empty or not required_columns.issubset(df.columns):
        print(f"Skipping {symbol} — missing data columns or no data.")
        continue

    df = generate_signals(df, vwma_period)
    num_signals = df['Position'].abs().sum()

    if num_signals == 0:
        print(f"Skipping {symbol} — no signals generated.")
        continue

    df = compute_simple_returns(df)
    plot_signals(df, symbol)
    plot_cumulative_returns(df, symbol)

    trade_log, final_capital = generate_trading_log(df, symbol, final_capital)

    if not trade_log.empty:
        all_logs.append(trade_log)
        trade_log.to_csv(os.path.join(output_dir, f"{symbol}_trade_log.csv"), index=False)
        print(f"Saved trade log for {symbol}")

print(all_logs)


Processing: SAIL.NS
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price                           Close        High         Low        Open  \
Datetime                                                                    
2025-06-10 03:45:00+00:00  134.270004  134.630005  134.149994  134.210007   

Price                      Volume  
Datetime                           
2025-06-10 03:45:00+00:00       0  
Saved trade log for SAIL.NS
[                       Date   Symbol Action   Price  Capital Before  \
0 2025-06-10 08:35:00+00:00  SAIL.NS    BUY  133.11        50000.00   
1 2025-06-10 08:45:00+00:00  SAIL.NS   SELL  132.87        50000.00   
2 2025-06-10 09:05:00+00:00  SAIL.NS    BUY  133.10        49999.76   
3 2025-06-10 09:15:00+00:00  SAIL.NS   SELL  133.00        49999.76   
4 2025-06-10 09:35:00+00:00  SAIL.NS    BUY  133.02        49999.66   
5 2025-06-10 09:55:00+00:00  SAIL.NS   SELL  132.81        49999.66   

   Capital After  
0       50000.00  
1       49999.76  
2       49999.76  
3       49999.66  
4       49999.66  
5       49999.45  ]